In [11]:
# Imports
import pandas as pd
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import load_model
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report

In [12]:
# Load the test dataset
data = pd.read_csv('UNSW_NB15_testing-set.csv')

In [13]:
# Select only the required features (same as training)
selected_features = [
    'dur', 'proto', 'service', 'state', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sload', 'dload',
    'spkts', 'dpkts', 'ct_srv_src', 'ct_srv_dst', 'smean', 'dmean', 'ct_dst_ltm', 'attack_cat'
]
data = data[selected_features]

In [15]:
data.ct_dst_ltm.unique().count()

AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [4]:
# Load the label encoders
with open('label_encoders.pkl', 'rb') as f:
    label_encoders = pickle.load(f)

# Apply label encoding with handling for unseen labels
for column, le in label_encoders.items():
    # Create a temporary column to avoid changing original data during transformation
    temp_column = data[column].copy()
    # Use the encoder with a default value for unseen labels
    data[column] = temp_column.apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)

In [5]:
# Separate features (X_test) and target (y_test)
X_test = data.drop(columns=['attack_cat'])
y_test = data['attack_cat']

In [6]:
# Separate features (X) and the target (y)
X = data.drop(columns=['attack_cat'])
y = data['attack_cat']
# Convert target to categorical if it’s multi-class
y_encoded = to_categorical(y)

In [7]:
# Normalize continuous features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
# Load the saved model
model = load_model('IDS_Detect.h5')

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Evaluate the model on the training dataset
y_pred = model.predict(X_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_encoded, axis=1)

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:")
print(cm)

# Print the classification report
report = classification_report(y_true, y_pred_classes)
print("Classification Report:")
print(report)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")


5480/5480 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step
Confusion Matrix:
[[    0     0   417  1012    49     4   446    72     0     0]
 [    0     0   371  1081    44    18    99   133     0     0]
 [    0     0  2824  7787   378   243   338   659    35     0]
 [    0     0  3842 24956   727   547  1854  1443    24     0]
 [    0     0   474  1249  2937   291 12738   470    25     0]
 [    0     0   112   432    18 39330    78    26     4     0]
 [    0     0    27   409  1115    98 54193   156     2     0]
 [    0     1   560  1630   302     4  1240  6754     0     0]
 [    0     0     9    92     4    22   313   479   214     0]
 [    0     0     0    86     8    26    10     0     0     0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2000
           1       0.00      0.00      0.00      1746
           2       0.33      0.23      0.27     12264
           3       0.64      0.75      0.69     33393
           4  

C:\Users\Admin\anaconda3\envs\VulDet\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Admin\anaconda3\envs\VulDet\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Admin\anaconda3\envs\VulDet\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │             170 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,932 (15.36 KB)

 Trainable params: 3,930 (15.35 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)